<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/generic_cypher_gpt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai neo4j

In [ ]:
#provide an LLM directly with schema information and let it construct Cypher statements based on graph schema information alone. 

In [31]:
node_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "node"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {labels: nodeLabels, properties: properties} AS output

"""

rel_properties_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE NOT type = "RELATIONSHIP" AND elementType = "relationship"
WITH label AS nodeLabels, collect(property) AS properties
RETURN {type: nodeLabels, properties: properties} AS output
"""

rel_query = """
CALL apoc.meta.data()
YIELD label, other, elementType, type, property
WHERE type = "RELATIONSHIP" AND elementType = "node"
RETURN {source: label, relationship: property, target: other} AS output
"""

In [32]:
from neo4j import GraphDatabase
from neo4j.exceptions import CypherSyntaxError
import openai


def schema_text(node_props, rel_props, rels):
    return f"""
  This is the schema representation of the Neo4j database.
  Node properties are the following:
  {node_props}
  Relationship properties are the following:
  {rel_props}
  Relationship point from source to target nodes
  {rels}
  Make sure to respect relationship types and directions
  """


class Neo4jGPTQuery:
    def __init__(self, url, user, password, openai_api_key):
        self.driver = GraphDatabase.driver(url, auth=(user, password))
        openai.api_key = openai_api_key
        # construct schema
        self.schema = self.generate_schema()


    def generate_schema(self):
        node_props = self.query_database(node_properties_query)
        rel_props = self.query_database(rel_properties_query)
        rels = self.query_database(rel_query)
        return schema_text(node_props, rel_props, rels)

    def refresh_schema(self):
        self.schema = self.generate_schema()

    def get_system_message(self):
        return f"""
        Task: Generate Cypher queries to query a Neo4j graph database based on the provided schema definition.
        Instructions:
        Use only the provided relationship types and properties.
        Do not use any other relationship types or properties that are not provided.
        If you cannot generate a Cypher statement based on the provided schema, explain the reason to the user.
        Schema:
        {self.schema}

        Note: Do not include any explanations or apologies in your responses.
        """

    def query_database(self, neo4j_query, params={}):
        with self.driver.session() as session:
            result = session.run(neo4j_query, params)
            output = [r.values() for r in result]
            output.insert(0, result.keys())
            return output

    def construct_cypher(self, question, history=None):
        messages = [
            {"role": "system", "content": self.get_system_message()},
            {"role": "user", "content": question},
        ]
        # Used for Cypher healing flows
        if history:
            messages.extend(history)

        completions = openai.ChatCompletion.create(
            model="gpt-4",
            temperature=0.0,
            max_tokens=1000,
            messages=messages
        )
        return completions.choices[0].message.content

    def run(self, question, history=None, retry=True):
        # Construct Cypher statement
        cypher = self.construct_cypher(question, history)
        print(cypher)
        try:
            return self.query_database(cypher)
        # Self-healing flow
        except CypherSyntaxError as e:
            # If out of retries
            if not retry:
              return "Invalid Cypher syntax"
        # Self-healing Cypher flow by
        # providing specific error to GPT-4
            print("Retrying")
            return self.run(
                question,
                [
                    {"role": "assistant", "content": cypher},
                    {
                        "role": "user",
                        "content": f"""This query returns an error: {str(e)} 
                        Give me a improved query that works without any explanations or apologies""",
                    },
                ],
                retry=False
            )


It's interesting how I ended with the final system message to get GPT-4 following my instructions. At first, I wrote my directions as plain text and added some constraints. However, the model wasn't doing exactly what I wanted, so I opened ChatGPT in a web browser and asked it to rewrite my instructions in a manner that GPT-4 would understand. Finally, ChatGPT seems to understand what works best as GPT-4 prompts, as the model behaved much better with this new prompt structure.


The GPT-4 model uses the ChatCompletion endpoint, which uses a combination of system, user, and optional assistant messages when we want to ask follow-up questions. So, we always start with only the system and user message. However, if the generated Cypher statement has any syntax error, the self-healing flow will be started, where we include the error in the follow-up question so that GPT-4 can fix the query. Therefore, we have included the optional history parameter for Cypher self-healing flow.

The run function starts by generating a Cypher statement. Then, the generated Cypher statement is used to query the Neo4j database. If the Cypher syntax is valid, the query results are returned. However, suppose there is a Cypher syntax error. In that case, we do a single follow-up to GPT-4, provide the generated Cypher statement it constructed in the previous call, and include the error from the Neo4j database. GPT-4 is quite good at fixing a Cypher statement when provided with the error.

The self-healing Cypher flow was inspired by others who have used similar flows for Python and other code. However, I have limited the follow-up Cypher healing to only a single iteration. If the follow-up doesn't provide a valid Cypher statement, the function returns the "Invalid Cypher syntax response".
Let's now test the capabilities of GPT-4 to construct Cypher statements based on the provided graph schema only.

In [1]:
import os
from IPython.display import clear_output

In [2]:
#env_vars = {
    "openai_key": "xxx"
}

In [3]:
#for key, value in env_vars.items():
    os.environ[key] = value

In [5]:
#openai_key = os.environ["openai_key"]

In [ ]:
 # create blank sandbox and seed the database with the following script.

In [33]:
astro_db = Neo4jGPTQuery(
    url="bolt://100.24.6.139:7687",
    user="neo4j",
    password="spray-hit-vision",
    openai_api_key=openai_key,
)

In [34]:
url = "https://raw.githubusercontent.com/lawrencerowland/Data-models-for-portfolios/master/Example-5-Using-Neo4j/Final_desktop_version.cypher"
#url="https://gist.githubusercontent.com/tomasonjo/52b2da916ef5cd1c2adf0ad62cc71a26/raw/a3a8716f7b28f3a82ce59e6e7df28389e3cb33cb/astro.cql"

In [ ]:
# the above runs once you take out the two constraint lines

In [35]:
astro_db.query_database("CALL apoc.cypher.runFile($url)", {'url':url})
astro_db.refresh_schema()

Failed to read from defunct connection IPv4Address(('100.24.6.139', 7687)) (ResolvedIPv4Address(('100.24.6.139', 7687)))


ServiceUnavailable: Failed to read from defunct connection IPv4Address(('100.24.6.139', 7687)) (ResolvedIPv4Address(('100.24.6.139', 7687)))

In [41]:
astro_db.run("""
what is the human_capital_management project improved by?
""")

To find the projects improved by the human_capital_management project, we can use the following Cypher query:

```cypher
MATCH (p1:Project {name: "human_capital_management"})-[:IMPROVED_BY]->(p2:Project)
RETURN p2.name as Improved_Projects
```

This query will return the names of the projects that are improved by the human_capital_management project. However, the provided schema does not have an "IMPROVED_BY" relationship type. Please provide the correct relationship type to generate a valid Cypher query.
Retrying
MATCH (p1:Project {name: "human_capital_management"})-[:IMPROVED_BY]->(p2:Project)
RETURN p2.name as Improved_Projects


[['Improved_Projects']]


* Multiple aggregations with different grouping keys are a problem
* Version two of the Graph Data Science library is beyond the knowledge cutoff date
* Sometimes it messes up the relationship direction (not frequently, though)
* The non-deterministic nature of GPT-4 makes it feel like you are dealing with a horoscope-based model, where identical queries work in the morning but not in the afternoon
* Sometimes the model bypasses system instructions and provides explanations for queries

Using the schema-only approach to GPT-4 can be used for experimental setups to help developers or researchers that don't have malicious intents to interact with the graph database. On the other hand, if you want to build something more production-ready, I would recommend using the approach of providing examples of Cypher statements.

NameError: name 'node_props' is not defined